In [1]:
# ---------------------------
# Imports
# ---------------------------
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
import os

# Import your CRUD module
from AnimalShelter import AnimalShelter  # Make sure your file is named AnimalShelter.py

# ---------------------------
# MongoDB Connection
# ---------------------------
username = "aacuser"
password = "SNHU1234"
db = AnimalShelter(username, password, port=27017)  # Use default MongoDB port

# Load initial data
try:
    df = pd.DataFrame.from_records(db.read({}))
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
except Exception as e:
    print("Error loading data:", e)
    df = pd.DataFrame()

# ---------------------------
# Initialize Dash App
# ---------------------------
app = JupyterDash(__name__)

# ---------------------------
# Load Logo
# ---------------------------
logo_path = "Grazioso Salvare logo.png"  # Make sure this file exists
if os.path.exists(logo_path):
    encoded_image = base64.b64encode(open(logo_path, 'rb').read()).decode()
    logo = html.A(
        html.Img(src=f"data:image/png;base64,{encoded_image}", style={'height':'150px'}),
        href="https://www.snhu.edu",
        target="_blank"
    )
else:
    logo = html.Div("Logo not found", style={'color':'red', 'fontSize':16})

# ---------------------------
# Layout
# ---------------------------
app.layout = html.Div([
    html.Center(html.H1("CS-340 Dashboard - Lenie Silverman")),
    logo,
    html.Hr(),

    # Filter Options
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Reset', 'value': 'RESET'},
            {'label': 'Water Rescue', 'value': 'WATER'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'MOUNTAIN'},
            {'label': 'Disaster/Individual Tracking', 'value': 'DISASTER'}
        ],
        value='RESET',
        labelStyle={'display': 'inline-block', 'margin-right':'15px'}
    ),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        filter_action="native",
        sort_action="native",
        row_selectable="single"
    ),
    html.Hr(),

    # Charts and Map
    html.Div(style={'display':'flex'}, children=[
        html.Div(id='graph-id', style={'width':'50%'}),
        html.Div(id='map-id', style={'width':'50%'})
    ])
])

# ---------------------------
# Callbacks
# ---------------------------
@app.callback(
    Output('datatable-id','data'),
    Input('filter-type','value')
)
def update_table(filter_type):
    query = {}
    if filter_type == 'WATER':
        query = {"animal_type": "Dog",
                 "breed": {"$in": ["Labrador Retriever Mix","Newfoundland","Chesapeake Bay Retriever"]},
                 "outcome_type": {"$ne":"Euthanasia"}}
    elif filter_type == 'MOUNTAIN':
        query = {"animal_type": "Dog",
                 "breed": {"$in": ["German Shepherd","Siberian Husky","Rottweiler","Alaskan Malamute","Old English Sheepdog"]},
                 "outcome_type": {"$ne":"Euthanasia"}}
    elif filter_type == 'DISASTER':
        query = {"animal_type": "Dog",
                 "breed": {"$in": ["Doberman Pinscher","Golden Retriever","Bloodhound","German Shepherd","Rottweiler"]},
                 "outcome_type": {"$ne":"Euthanasia"}}
    
    df_filtered = pd.DataFrame.from_records(db.read(query))
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

@app.callback(
    Output('graph-id','children'),
    Input('datatable-id','derived_virtual_data')
)
def update_pie(viewData):
    if viewData is None or len(viewData)==0:
        return html.Div("No data available")
    df_chart = pd.DataFrame(viewData)
    fig = px.pie(df_chart, names='breed', title='Dogs by Breed')
    return dcc.Graph(figure=fig)

@app.callback(
    Output('map-id','children'),
    [Input('datatable-id','derived_virtual_data'),
     Input('datatable-id','selected_rows')]
)
def update_map(viewData, selected_rows):
    if viewData is None or len(viewData)==0:
        return []
    df_map = pd.DataFrame(viewData)
    row = selected_rows[0] if selected_rows else 0
    lat = df_map.iloc[row]['location_lat']
    lon = df_map.iloc[row]['location_long']
    name = df_map.iloc[row]['name'] if df_map.iloc[row]['name'] else "No Name"
    breed = df_map.iloc[row]['breed']
    
    return dl.Map(
        style={'width':'100%','height':'500px'},
        center=[lat, lon],
        zoom=10,
        children=[
            dl.TileLayer(),
            dl.Marker(
                position=[lat, lon],
                children=[
                    dl.Tooltip(breed),
                    dl.Popup(html.Div([html.H4("Animal Name"), html.P(name)]))
                ]
            )
        ]
    )

# ---------------------------
# Run Server
# ---------------------------
if __name__ == "__main__":
    app.run_server(mode="jupyterlab", port=8054, debug=True)


MongoDB connected successfully!
Read failed: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}
